# DeepDream Pretrained ResNet50 on MIT Indoor Scene Recognition
- Reference:
  - DeepDream Algorithm: https://ai.googleblog.com/2015/06/inceptionism-going-deeper-into-neural.html

## 1. What is DeepDream?
DeepDream is an artistic algorithm where a pretrained CNN is fed an image and optimized to amplify the features it "sees" in the image

To achieve this, we can do a gradient ascent usually on the MSE loss constructed over the activations coming from a certain layer. The MSE is constructed between those activations and the all-0s tensor, which would by itself push the highest activations strongest towards zero, but by switching the sign, and doing a gradient ascent instead of descent, we end up amplifying them the most.

Depending on the neural network layer the features amplified will either be low level (such as edges, or certain geometric patterns) or high level that heavily depends on the dataset on which the network was pretrained.

## 2. Coding

### 2.1. Import Library

In [8]:
# Python native libraries
import os
import enum
from collections import namedtuple
import argparse
import numbers
import math

# Deep learning related imports
import torch
import torch.nn as nn
from torchvision import models
from torchvision import transforms
import torch.nn.functional as F
from torch.hub import download_url_to_file
from torch.utils.data import random_split
from torch.utils.data.dataloader import DataLoader
from torchvision.datasets import ImageFolder

import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt 

### 2.2. Prepare Device

In [2]:
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
    
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

In [3]:
device = get_default_device()
device

device(type='cpu')

### 2.3. Prepare Dataset

In [4]:
!ls ../../data/indoorCVPR_09/Images

airport_inside	computerroom	     inside_subway   pantry
artstudio	concert_hall	     jewelleryshop   poolinside
auditorium	corridor	     kindergarden    prisoncell
bakery		deli		     kitchen	     restaurant
bar		dentaloffice	     laboratorywet   restaurant_kitchen
bathroom	dining_room	     laundromat      shoeshop
bedroom		elevator	     library	     stairscase
bookstore	fastfood_restaurant  livingroom      studiomusic
bowling		florist		     lobby	     subway
buffet		gameroom	     locker_room     toystore
casino		garage		     mall	     trainstation
children_room	greenhouse	     meeting_room    tv_studio
church_inside	grocerystore	     movietheater    videostore
classroom	gym		     museum	     waitingroom
cloister	hairsalon	     nursery	     warehouse
closet		hospitalroom	     office	     winecellar
clothingstore	inside_bus	     operating_room


In [6]:
data_dir = '../../data/indoorCVPR_09/Images'
classes = os.listdir(data_dir)
print(f"There are {len(classes)} classes: \n {classes}")

There are 67 classes: 
 ['videostore', 'warehouse', 'bookstore', 'operating_room', 'meeting_room', 'mall', 'bar', 'auditorium', 'kitchen', 'restaurant', 'fastfood_restaurant', 'gym', 'bathroom', 'subway', 'closet', 'hospitalroom', 'clothingstore', 'casino', 'gameroom', 'restaurant_kitchen', 'nursery', 'grocerystore', 'shoeshop', 'movietheater', 'bowling', 'bedroom', 'corridor', 'poolinside', 'children_room', 'hairsalon', 'dining_room', 'classroom', 'jewelleryshop', 'laboratorywet', 'elevator', 'stairscase', 'concert_hall', 'winecellar', 'tv_studio', 'garage', 'airport_inside', 'florist', 'artstudio', 'prisoncell', 'computerroom', 'library', 'museum', 'waitingroom', 'toystore', 'lobby', 'buffet', 'church_inside', 'deli', 'office', 'dentaloffice', 'bakery', 'cloister', 'studiomusic', 'livingroom', 'kindergarden', 'laundromat', 'pantry', 'inside_subway', 'trainstation', 'locker_room', 'inside_bus', 'greenhouse']


In [9]:
# Transform images to fit the neural network
transformations = transforms.Compose([transforms.Resize((256, 256)), transforms.ToTensor()])

dataset = ImageFolder(data_dir, transform = transformations)

In [11]:
# Load and split dataset
random_seed = 42
torch.manual_seed(random_seed)

train_set, val_set, test_set = random_split(dataset, [13000, 2000, 620])
len(train_set), len(val_set), len(test_set)

(13000, 2000, 620)

In [12]:
# Convert validation dataset into DataLoader to the model
class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

In [13]:
batch_size = 25

val_dl = DataLoader(val_set, batch_size, num_workers = 4, pin_memory=True)
val_dl = DeviceDataLoader(val_dl, device)

### 2.4. Initialize the 